In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('LifeTrack_white_stork_clean.csv')

In [3]:
print('ile jest opasek: ')
print(df['tag-local-identifier'].nunique())

ile jest opasek: 
24


In [4]:
# kolumna poszczegolnych przelotow
# 1 na koncu id oznacza pl->afr (najczesciej jest to sierpien-styczen)
# 2 na koncu id oznacza afr->pl ( najczesciej to kwiecien - sierpien)
# data graniczna to 1.02.2014
df['road_id'] = None 

In [5]:
for tag_id in df['tag-local-identifier'].unique():
    condition = (df['tag-local-identifier'] == tag_id)
    
    condition_1 = (pd.to_datetime(df['date']) < pd.to_datetime("2014-02-01"))
    condition_2 = ~condition_1

    df.loc[condition & condition_1, 'road_id'] = f'{tag_id}1'
    df.loc[condition & condition_2, 'road_id'] = f'{tag_id}2'

In [6]:
df.to_csv('road_id_stork.csv', index=False)

In [7]:
df['date'] = pd.to_datetime(df['date'])
max_time_diff_row = df.groupby('road_id')['date'].agg(lambda x: x.max() - x.min()).idxmax()
print('road id z najwieksza liczba punktow')
print(max_time_diff_row)

road id z najwieksza liczba punktow
30482


In [8]:
min_rows_tag = df['road_id'].value_counts().idxmin()
min_rows_count = df['road_id'].value_counts().min()
print('najmniejsza liczba punktow przelotu')
print(min_rows_count)

najmniejsza liczba punktow przelotu
3704


In [9]:
max_rows_tag = df['road_id'].value_counts().idxmax()
max_rows_count = df['road_id'].value_counts().max()
print('najwieksza liczba punktow przelotu')
print(max_rows_count)

najwieksza liczba punktow przelotu
103046


In [10]:
min_time_diff_row = df.groupby('road_id')['date'].agg(lambda x: (x.max() - x.min()).total_seconds()).idxmin()
min_time_diff = (df.groupby('road_id')['date'].agg(lambda x: (x.max() - x.min()).total_seconds())).min()

print(f'Road id z najkrótszym czasem: {min_time_diff_row}')
print(f'Najkrótszy czas dla tego Road id: {min_time_diff} sekund')
min_time_diff_days = min_time_diff / (24 * 60 * 60)
print(f'{min_time_diff_days} dni')

Road id z najkrótszym czasem: 45712
Najkrótszy czas dla tego Road id: 1453343.0 sekund
16.821099537037036 dni


In [17]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

first_row = df.iloc[0]
first_timestamp = first_row['timestamp']

interval = pd.to_timedelta('1 hour')

filtered_rows = [first_row]

for _, row in df.iterrows():
    current_timestamp = row['timestamp']
    time_diff = current_timestamp - first_timestamp

    if time_diff >= interval:
        filtered_rows.append(row)
        first_timestamp = current_timestamp
        
filtered_df = pd.DataFrame(filtered_rows)

          event-id           timestamp  location-long  location-lat  \
0        239769795 2013-07-16 12:04:53      15.637698     52.092062   
12       239769807 2013-07-16 13:05:07      15.637515     52.092192   
24       239769819 2013-07-16 14:05:07      15.637412     52.092132   
36       239769831 2013-07-16 15:05:07      15.637464     52.092129   
48       239769843 2013-07-16 16:05:23      15.637632     52.092164   
...            ...                 ...            ...           ...   
437609   994120696 2015-09-15 18:00:06      29.633623     13.219532   
437614   999216476 2015-09-17 18:00:08      29.422373     13.046986   
437619  1002230321 2015-09-19 18:00:23      29.190209     12.375587   
437624  1002610711 2015-09-20 18:00:39      29.233074     12.355011   
437629  1005453317 2015-09-21 18:00:08      28.342386     11.722287   

        ground-speed  height-above-ellipsoid individual-taxon-canonical-name  \
0               0.03                   127.3                 Ciconi

In [18]:
filtered_df

,event-id,timestamp,location-long,location-lat,ground-speed,height-above-ellipsoid,individual-taxon-canonical-name,eobs:temperature,tag-local-identifier,individual-local-identifier,date,road_id
0,239769795,2013-07-16 12:04:53,15.637698,52.092062,0.03,127.3,Ciconia ciconia,34.0,3048,PLG 5P311 (3048),2013-07-16 12:04:53,30481
12,239769807,2013-07-16 13:05:07,15.637515,52.092192,0.46,128.6,Ciconia ciconia,32.0,3048,PLG 5P311 (3048),2013-07-16 13:05:07,30481
24,239769819,2013-07-16 14:05:07,15.637412,52.092132,0.55,125.3,Ciconia ciconia,30.0,3048,PLG 5P311 (3048),2013-07-16 14:05:07,30481
36,239769831,2013-07-16 15:05:07,15.637464,52.092129,0.12,133.3,Ciconia ciconia,34.0,3048,PLG 5P311 (3048),2013-07-16 15:05:07,30481
48,239769843,2013-07-16 16:05:23,15.637632,52.092164,0.10,134.2,Ciconia ciconia,32.0,3048,PLG 5P311 (3048),2013-07-16 16:05:23,30481
...,...,...,...,...,...,...,...,...,...,...,...,...
437609,994120696,2015-09-15 18:00:06,29.633623,13.219532,NaN,NaN,Ciconia ciconia,NaN,3955,"Filippa + PLG VT1357 (GPS3955, ECG3479)",2015-09-15 18:00:06,39552
437614,999216476,2015-09-17 18:00:08,29.422373,13.046986,NaN,NaN,Ciconia ciconia,NaN,3955,"Filippa + PLG VT1357 (GPS3955, ECG3479)",2015-09-17 18:00:08,39552
437619,1002230321,2015-09-19 18:00:23,29.190209,12.375587,NaN,NaN,Ciconia ciconia,NaN,3955,"Filippa + PLG VT1357 (GPS3955, ECG3479)",2015-09-19 18:00:23,39552
437624,1002610711,2015-09-20 18:00:39,29.233074,12.355011,NaN,NaN,Ciconia ciconia,NaN,3955,"Filippa + PLG VT1357 (GPS3955, ECG3479)",2015-09-20 18:00:39,39552
